In [5]:
# Import the relevant classes
from classes_EIS import DataHandler, RRCRCModel,RRCRCCPEModel, FitManager

# Example of how to fit multiple models with varying parameter numbers
data_handler = DataHandler('MPR files/1_00V_10cyc.mpr')

print (data_handler.Ewe)

# Define initial guesses and bounds for each 
Rs_guess = 11.
R_guess = 1.5
fs_guess = 10**1.3
n_guess = 1.
R2_guess = 6e4
fs2_guess = 10**0.5
n2_guess = 1.

initial_guess_rrcrc = [Rs_guess, R_guess, 3.2e-5, R2_guess, 1.2e-4]
bounds_rrcrc = ([5, 1, 1e-7, 1, 1e-7], [15, 5, 1e-3, 1e6, 1e-3])

initial_guess_rrcrccpe = [Rs_guess, R_guess, fs_guess,n_guess, R2_guess, fs2_guess,n2_guess]
bounds_rrcrccpe = ([5, 1, 1e-2, 0.5, 1, 1e-2, 0.5], [15, 5, 1e7, 1.5, 1e6, 1e7, 1.5])

# Initialize models with dynamic number of parameters
rrcrc_model = RRCRCModel(initial_guess_rrcrc, bounds_rrcrc)
rrcrccpe_model = RRCRCCPEModel(initial_guess_rrcrccpe, bounds_rrcrccpe)

# Initialize the fit manager
fit_manager = FitManager(data_handler)

# Fit both models
fitted_rrcrc_model, rrcrc_covariance, rrcrc_fit_quality = fit_manager.fit_model(rrcrc_model, fmin=4e3, fmax=1e6)
fitted_rrcrccpe_model, rrcrccpe_covariance, rrcrccpe_fit_quality = fit_manager.fit_model(rrcrccpe_model, fmin=4e3, fmax=1e6)

# Print optimized parameters
print("Optimized RRCRC Model Parameters:", fitted_rrcrc_model.params)
print("Fit Quality Metrics for RRCRC Model:", rrcrc_fit_quality)
print("Optimized RRCRCCPE Model Parameters:", fitted_rrcrccpe_model.params)
print("Fit Quality Metrics for RRCRCCPE Model:", rrcrccpe_fit_quality)



1.000628
Optimized RRCRC Model Parameters: [1.01091240e+01 1.07459792e+00 3.26489827e-06 1.78702566e+00
 7.47721017e-05]
Fit Quality Metrics for RRCRC Model: {'MSE': np.float64(0.0004849272163225992), 'RMSE': np.float64(0.02202106301527243), 'R-squared': np.float64(0.9999854809569958)}
Optimized RRCRCCPE Model Parameters: [1.07549992e+01 1.00000000e+00 1.15423176e+05 9.54953030e-01
 1.17263899e+00 1.29750250e+04 1.11701287e+00]
Fit Quality Metrics for RRCRCCPE Model: {'MSE': np.float64(0.0001778294074020061), 'RMSE': np.float64(0.0133352693036926), 'R-squared': np.float64(0.9999946756694065)}
